In [ ]:
#Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb
import sklearn as skl
from datetime import datetime
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import TimeSeriesSplit
plt.style.use('fivethirtyeight')
color_pal = sns.color_palette()

In [ ]:
#Reading data + Indexing
df = pd.read_csv("PJME_hourly.csv")
df = df.set_index("Datetime")
df.index = pd.to_datetime(df.index)

In [ ]:
#Plot the whole data 
df.plot(style='.',
        figsize=(15,5),
        color=color_pal[0],
        title='PJME usage in MW')
plt.show()

In [ ]:
#Outlier Removal
df = df.query('PJME_MW > 19_000').copy()


In [ ]:
# Feature Creation
def create_features(df):
    """
    Create Features
    """
    df = df.copy()
    df["year"] = df.index.year
    df["quarter"] = df.index.quarter
    df["month"] = df.index.month
    df["day_of_week"] = df.index.weekday
    df["day_of_month"] = df.index.day
    df["day_of_year"] = df.index.dayofyear
    df["hour"] = df.index.hour
    return df

df = create_features(df)

In [ ]:
# Lag Features   # Setting up 364 day max lag makes so that we can only predict 364 days into the future
def add_lags(df):
    target_map = df['PJME_MW'].to_dict()
    df['lag1'] = (df.index - pd.Timedelta('364 days')).map(target_map)
    df['lag2'] = (df.index - pd.Timedelta('728 days')).map(target_map)
    df['lag3'] = (df.index - pd.Timedelta('1092 days')).map(target_map)
    return df

df = add_lags(df)

In [ ]:
# Train/Test Split
train = df.loc[df.index < "01-01-2016"]
test = df.loc[df.index >= "01-01-2016"]

fig, ax = plt.subplots(figsize=(15,5))
train['PJME_MW'].plot(ax=ax,
           label = "Training Set",
           title = "Train/Slipt"
        )
test['PJME_MW'].plot(ax=ax,
           label = "Test Set"
        )
ax.axvline("01-01-2016",
          color = "black",
          ls = "--"
           )
ax.legend(["Training Set","Test Set"])
plt.show()

In [ ]:
#Create Model
train = create_features(train)
test = create_features(test)
FEATURES = ['hour','day_of_year','day_of_month','day_of_week','month','quarter','year','lag1','lag2','lag3']
TARGET = 'PJME_MW'

In [ ]:
#Visualizing the Data (Day of the Month)
fig, ax = plt.subplots(figsize=(20, 8))
sns.boxplot(data=df,
            x='hour',
            y='PJME_MW'
            )
ax.set_title("Energy Usage per Hour")
plt.show()

In [ ]:
#Visualizing the Data (Day of the Week)
fig, ax = plt.subplots(figsize=(20, 8))
sns.boxplot(data=df,
            x='day_of_week',
            y='PJME_MW'
            )
ax.set_title("Energy Usage per Day of the Week")
plt.show()

In [ ]:
#Visualizing the Data (Day of the Month)
fig, ax = plt.subplots(figsize=(20, 8))
sns.boxplot(data=df,
            x='day_of_month',
            y='PJME_MW'
            )
ax.set_title("Energy Usage per Day of the Month")
plt.show()

In [ ]:
#Visualizing the Data (Month)
fig, ax = plt.subplots(figsize=(20, 8))
sns.boxplot(data=df,
            x='month',
            y='PJME_MW'
            )
ax.set_title("Energy Usage per Month")
plt.show()

In [ ]:
#Visualizing the Data (Quarter)
fig, ax = plt.subplots(figsize=(20, 8))
sns.boxplot(data=df,
            x='quarter',
            y='PJME_MW'
            )
ax.set_title("Energy Usage per Quarter")
plt.show()

In [ ]:
x_train = train[FEATURES]
y_train = train[TARGET]

x_test = test[FEATURES]
y_test = test[TARGET]

In [ ]:
#Hyperparameter Tunning
    #Here is where we can tune our model to make it as efficient as it can be (If we select too many options it could take a very long time to run cause of the size of our dataset)
def hyperParameterTuning(x_train, y_train):
    param_tuning = {
        'booster':['gbtree','gblinear'],
        'tree_method':['exact','hist','approx'],
        #'max_bin': [300, 400, 500],
        'learning_rate': [0.1, 1],
        #'max_depth': [3, 8, 9],
        #'min_child_weight': [3, 8, 10],
        #'subsample': [0.1, 0.3, 0.5],
        #'colsample_bytree': [0.1, 0.3, 0.5],
        'n_estimators' : [400, 500],
        'objective': ['reg:squarederror', 'reg:linear']
    }

    reg = xgb.XGBRegressor()

    gsearch = GridSearchCV(estimator = reg,
                           param_grid = param_tuning,                        
                           #scoring = 'neg_mean_absolute_error', #MAE
                           #scoring = 'neg_mean_squared_error',  #MSE
                           cv = 5,
                           n_jobs = -1,
                           verbose = 1)

    gsearch.fit(x_train,y_train)

    return gsearch.best_params_

In [ ]:
#hyperParameterTuning(x_train,y_train)

In [ ]:
# Best Fit
        #Here we run the best setting we get from the hyperparameter tunning fuction and compare it with the testing data
        #Also saves the last run in a .json file so we can load it again
reg = xgb.XGBRegressor(base_score=0.5,
                       objective='reg:linear',
                       booster='gbtree',
                       tree_method ='hist',
                       n_estimators = 1000,
                       colsample_bytree = 0.1,
                       max_depth = 8,
                       min_child_weight = 0,
                       max_bin = 300,
                       learning_rate = 0.01,
                       early_stopping_rounds = 50,
                       subsample = 0.5
                       )
reg.fit(x_train, y_train,
        eval_set = [(x_train, y_train), (x_test, y_test)],
        verbose = 100)

reg.save_model('last_run.json')

In [ ]:
#Feature Importance
    #How much of each feature was taken in consideration when trainning the model
fi = pd.DataFrame(data=reg.feature_importances_,
             index=reg.feature_names_in_,
             columns=['Importance']
)
fi.sort_values('Importance').plot(kind='barh',
                                  title='Feature Importance')
plt.show()

In [ ]:
#Creates the predictions values
    #Saves it on a .csv file
test['prediction'] = reg.predict(x_test)
df = df.merge(test[['prediction']], how = 'left', left_index = True, right_index = True)
df.to_csv(r'G:\Meu Drive\Documents\Currículo\Portfolio\Energy Consumption Forecast\result.csv')

In [ ]:
#Plot the comparison between the actuals and the predictions
ax = df[['PJME_MW']].plot(figsize = (20,8))
df["prediction"].plot(ax=ax, style = '-')
plt.legend(['Truth Data','Predictions'])
ax.set_title('Raw Data and Predictions')
plt.show()

In [ ]:
#RMSE Score
    #This is used to score our model, improving the model should make this number lower
score = np.sqrt(mean_squared_error(test['PJME_MW'], test['prediction']))
print(f'RMSE Score on test set: {score:0.2f}')

In [ ]:
test['error'] = np.abs(test[TARGET]-test['prediction'])

In [ ]:
#Worst Predictions
test.groupby(['Datetime'])['error'].mean().sort_values(ascending=False).head(5)

In [ ]:
#Best Predictions
test.groupby('Datetime')['error'].mean().sort_values(ascending=True).head(5)

In [ ]:
#Now we will train the model again, but this time with the whole dataset in order to predict into the future
df = create_features(df)
FEATURES = ['hour','day_of_year','day_of_month','day_of_week','month','quarter','year','lag1','lag2','lag3']
TARGET = 'PJME_MW'

x_all = df[FEATURES]
y_all = df[TARGET]

In [ ]:
#Second trainning
reg = xgb.XGBRegressor(base_score=0.5,
                       objective='reg:linear',
                       booster='gbtree',
                       tree_method ='hist',
                       n_estimators = 1000,
                       colsample_bytree = 0.1,
                       max_depth = 8,
                       min_child_weight = 0,
                       max_bin = 300,
                       learning_rate = 0.01,
                       early_stopping_rounds = 50,
                       subsample = 0.5
                       )
reg.fit(x_all, y_all,
        eval_set = [(x_all, y_all)],
        verbose = 100)

reg.save_model('last_run.json')

In [ ]:
#Create future dataframe
future = pd.date_range('2018-01-03','2019-08-01', freq='1h')
future_df = pd.DataFrame(index = future)
future_df['isFuture'] = True
df['isFuture'] = False
df_and_future = pd.concat([df, future_df])
df_and_future = create_features(df_and_future)
df_and_future = add_lags(df_and_future)

In [ ]:
#Gets only the future part of the dataframe
futue_w_features = df_and_future.query('isFuture').copy()

In [ ]:
#Predict the future
futue_w_features['pred'] = reg.predict(futue_w_features[FEATURES])

In [ ]:
#Plot the Future Predictions
futue_w_features['pred'].plot(figsize = (10,5),
                              title = 'Future Predictions')
plt.show()